# Pasul 2 – Colectarea și popularea bazei de date (Date EIA)

**Autor:** Cătălina Minciună  
**Curs:** Fluxuri de Date pentru Știința Datelor (ADC 2025)

**Scop:** Descărcarea și normalizarea datelor energetice din EIA API (EIA v2) pentru un set de țări și indicatori, în perioada 2

## Context și unitatea de analiză

În cadrul proiectului, datele EIA descriu indicatori energetici anuali care vor fi combinați ulterior cu indicatori socio-economici (WDI).  
Unitatea de analiză utilizată este **țară × an** (countryRegionId × year), iar fiecare indicator este identificat prin `productId`.


## Sursa datelor și parametrii extrași

**Sursă:** EIA API v2 – endpoint: `https://api.eia.gov/v2/international/data/`  
**Autentificare:** API key stocat local în `credentials.py` (nu este inclus în repository)

**Țări (ISO3 / countryRegionId):** ROU, BGR, HUN, POL, CZE, SVK  
**Indicatori (productId):**
- `44` – Primary energy  
- `47` – Energy intensity  
- `4008` – CO₂ emissions  

**Frecvență:** annual  
**Perioadă cerută:** 2019–2024 (în funcție de disponibilitatea seriilor în EIA

## Import librării și configurare

Inițializez bibliotecile necesare și parametrii de descărcare. Cheia API este importată dintr-un fișier local `credentials.py`.

In [25]:
import requests
import pandas as pd
from credentials import eia_api_key

In [ ]:
# Configurare
URL = "https://api.eia.gov/v2/international/data/"
COUNTRIES = ["ROU", "BGR", "HUN", "POL", "CZE", "SVK"]
PRODUCT_IDS = [44, 47, 4008]  # Primary energy, Energy intensity, CO2 emissions
START = "2019"
END = "2024"

## Funcție reutilizabilă pentru descărcarea datelor (per indicator)

Pentru a evita cod duplicat, folosesc o funcție care:
1. face request către EIA API pentru un `productId`
2. aplică filtrele de bază (ani, frecvență, țări)
3. normalizează răspunsul JSON în tabel (pandas DataFrame)
4. păstrează doar coloanele relevante pentru etapa de normalizare (Pasul 2)

In [ ]:

def fetch_eia_product(product_id: int) -> pd.DataFrame:
    """Fetch data for a single product from EIA API"""
    params = {
        "api_key": eia_api_key,
        "frequency": "annual",
        "data[]": "value",
        "start": START,
        "end": END,
        "offset": 0,
        "length": 5000,
        "facets[productId][]": str(product_id),
    }
    
    # Adăugăm toate țările analizate ca filtre
    for c in COUNTRIES:
        key = f"facets[countryRegionId][]"
        if key not in params:
            params[key] = []
        params[key].append(c)
    
    r = requests.get(URL, params=params, timeout=60)
    r.raise_for_status()
    payload = r.json()
    
    rows = payload.get("response", {}).get("data", [])
    df = pd.json_normalize(rows)
    
    if df.empty:
        return pd.DataFrame(columns=["countryRegionId", "period", "productId", "value", "unit", "activityId"])
    
    # Păstrăm coloanele relevante pentru normalizare
    keep = [c for c in ["countryRegionId", "period", "productId", "value", "unit", "activityId"] if c in df.columns]
    df = df[keep].copy()
    
    return df

## Descărcarea datelor EIA și consolidarea dataset-ului

Datele sunt descărcate separat pentru fiecare `productId` și apoi concatenate într-un singur tabel (`eia_raw`).  
În această etapă nu se aplică selecția unei singure unități sau eliminarea seriilor paralele; acestea vor fi tratate în Pasul 3.

In [28]:
print("Fetching data from EIA API...")
dfs = [fetch_eia_product(pid) for pid in PRODUCT_IDS]
eia_raw = pd.concat(dfs, ignore_index=True)


Fetching data from EIA API...


In [29]:
# 1. Extrage anul din câmpul 'period' (ex: "2023" -> 2023)
eia_raw["year"] = eia_raw["period"].astype(str).str[:4].astype(int)

# 2. Convertește value la numeric
eia_raw["value"] = pd.to_numeric(eia_raw["value"], errors="coerce")

# 3. Convertește productId la int
eia_raw["productId"] = eia_raw["productId"].astype(int)

# 4. Selectează și reordonează coloanele în format tabelar standard
# Fiecare rând = o observație unică (țară, an, indicator)
eia_normalized = eia_raw[[
    "countryRegionId",  # Țara
    "year",             # Anul
    "productId",        # ID indicator energetic
    "value",            # Valoarea
    "unit",             # Unitatea de măsură
    "activityId"        # ID activitate (metadata)
]].copy()

# 5. Filtrare: păstrează doar perioada 2019-2024
eia_normalized = eia_normalized[
    (eia_normalized["year"] >= 2019) & 
    (eia_normalized["year"] <= 2024)
]

# 6. Sortare pentru organizare clară
eia_normalized = eia_normalized.sort_values(
    by=["countryRegionId", "year", "productId"]
).reset_index(drop=True)


In [30]:
print("\n" + "="*50)
print("NORMALIZARE COMPLETĂ")
print("="*50)


NORMALIZARE COMPLETĂ


## Normalizare (format „tidy/long”)

Transform datele în format tabelar standard (tidy data), unde:
- fiecare rând reprezintă o observație (țară, an, indicator)
- coloanele principale sunt: `countryRegionId`, `year`, `productId`, `value`
- păstrez și metadate utile: `unit`, `activityId`

Pașii de normalizare:
1. extrag `year` din `period`
2. convertesc `value` la numeric
3. convertesc `productId` la int
4. selectez și reordonez coloanele finale
5. filtrez intervalul 2019–2024
6. sortez pentru consistență

In [31]:
print(f"\nShape: {eia_normalized.shape}")
print(f"Perioada: {eia_normalized['year'].min()} - {eia_normalized['year'].max()}")
print(f"Țări: {sorted(eia_normalized['countryRegionId'].unique())}")
print(f"Indicatori (productId): {sorted(eia_normalized['productId'].unique())}")




Shape: (216, 6)
Perioada: 2020 - 2023
Țări: ['BGR', 'CZE', 'HUN', 'POL', 'ROU', 'SVK']
Indicatori (productId): [np.int64(44), np.int64(47), np.int64(4008)]


## Verificări rapide (calitate și consistență)

Verific:
- dimensiunea dataset-ului
- intervalul de ani disponibil (poate diferi de perioada cerută, în funcție de completitudinea seriei)
- țările și indicatorii prezenți
- valori lipsă
- duplicate pe cheia (țară, an, productId)

**Notă despre duplicate:**  
EIA poate furniza același indicator în multiple unități (ex. MTOE, QBTU, TJ) sau în serii diferite.  
La Pasul 2, aceste observații sunt păstrate pentru transparență; în Pasul 3 se va selecta o singură unitate/serie standard per indicator pentru a obține o observație unică per țară–an–indicator.

In [32]:
# Verifică structura datelor
print("\n--- Structura datelor ---")
print(eia_normalized.info())


--- Structura datelor ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   countryRegionId  216 non-null    object 
 1   year             216 non-null    int64  
 2   productId        216 non-null    int64  
 3   value            216 non-null    float64
 4   unit             216 non-null    object 
 5   activityId       216 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 10.3+ KB
None


In [33]:
# Verifică valori lipsă
print("\n--- Valori lipsă ---")
print(eia_normalized.isnull().sum())


--- Valori lipsă ---
countryRegionId    0
year               0
productId          0
value              0
unit               0
activityId         0
dtype: int64


In [34]:
# Verifică duplicate (ar trebui să fie 0 după normalizare)
dups = eia_normalized.duplicated(subset=["countryRegionId", "year", "productId"]).sum()
print(f"\nDuplicate pe (țară, an, productId): {dups}")



Duplicate pe (țară, an, productId): 144


In [35]:
# Preview
print("\n--- Preview date normalizate (primele 15 rânduri) ---")
print(eia_normalized.head(15))



--- Preview date normalizate (primele 15 rânduri) ---
   countryRegionId  year  productId          value       unit activityId
0              BGR  2020         44       9.417170       MTOE          1
1              BGR  2020         44       0.373704       QBTU          1
2              BGR  2020         44  394278.081589         TJ          1
3              BGR  2020         44      16.681647       MTOE          2
4              BGR  2020         44       0.661981       QBTU          2
5              BGR  2020         44  698427.203784         TJ          2
6              BGR  2020         47     101.126811     MBTUPP         33
7              BGR  2020         47       4.347278  TBTUUSDPP         34
8              BGR  2020       4008      35.177193      MMTCD          8
9              BGR  2021         44      10.829147       MTOE          1
10             BGR  2021         44       0.429735       QBTU          1
11             BGR  2021         44  453394.711454         TJ        

## Salvarea datelor
Rezultatul acestui notebook este un singur fișier CSV, în format long (normalizat), pregătit pentru încărcare în baza de date și pentru combinarea ulterioară cu datele WDI.

Fișier generat:
- `01_eia_data_catalinaminciuna.csv`

In [ ]:
output_file = "01_eia_data_catalinaminciuna.csv"
eia_normalized.to_csv(output_file, index=False)